# Migración MySQL → MongoDB (sin SQLite)

Usa `.env` compartido (Node/Python).

## Requisitos
```bash
pip install pymysql pymongo python-dotenv
```

## Cargar `.env`

In [ ]:
from dotenv import load_dotenv
import os
ENV_PATH=os.environ.get("ENV_PATH",".env")
load_dotenv(dotenv_path=ENV_PATH, override=False)
print("[INFO] DB_ENV=", os.getenv("DB_ENV","local"))

## Resolver configuración

In [ ]:
def pick_sql_config():
    env=(os.getenv("DB_ENV","local") or "local").lower()
    if env=="remote":
        return {"database":os.getenv("DB_REMOTE_NAME"),"username":os.getenv("DB_REMOTE_USER"),"password":os.getenv("DB_REMOTE_PASSWORD"),"host":os.getenv("DB_REMOTE_HOST","localhost"),"port":int(os.getenv("DB_REMOTE_PORT","3306")),"dialect":os.getenv("DB_REMOTE_DIALECT","mysql")}
    else:
        return {"database":os.getenv("DB_LOCAL_NAME"),"username":os.getenv("DB_LOCAL_USER"),"password":os.getenv("DB_LOCAL_PASSWORD"),"host":os.getenv("DB_LOCAL_HOST","localhost"),"port":int(os.getenv("DB_LOCAL_PORT","3306")),"dialect":os.getenv("DB_LOCAL_DIALECT","mysql")}

def pick_mongo_uri():
    env=(os.getenv("DB_ENV","local") or "local").lower()
    return os.getenv("MONGODB_REMOTE_URI","") if env=="remote" else os.getenv("MONGODB_LOCAL_URI","")

SQL_CONF=pick_sql_config()
MONGO_URI=pick_mongo_uri()
MONGO_DB=os.getenv("MONGO_DB","apiheroes_nosql")
{ "sql": {k:(v if k!="password" else "***") for k,v in SQL_CONF.items()}, "mongo_uri_defined": bool(MONGO_URI), "mongo_db": MONGO_DB }

## Dependencias + utilidades

In [ ]:
import os, json
from datetime import datetime
import pymysql
try:
    from pymongo import MongoClient, ReplaceOne
except Exception:
    MongoClient=None; ReplaceOne=None

def to_bool(x):
    if isinstance(x,bool): return x
    if x is None: return False
    if isinstance(x,(int,float)): return x!=0
    return str(x).strip().lower() in {"1","true","t","yes","y","on"}

def to_iso_date(x):
    if x in (None,"","0000-00-00"): return None
    for fmt in ("%Y-%m-%d","%Y-%m-%d %H:%M:%S"):
        try: return datetime.strptime(str(x),fmt).date().isoformat()
        except: pass
    try: return datetime.fromisoformat(str(x)).date().isoformat()
    except: return str(x)

## Extraer desde MySQL

In [ ]:
if (SQL_CONF.get("dialect","mysql") or "mysql").lower()!="mysql":
    raise RuntimeError(f"Dialect no soportado: {SQL_CONF.get('dialect')}")
conn=pymysql.connect(host=SQL_CONF["host"],port=int(SQL_CONF["port"]),user=SQL_CONF["username"],password=SQL_CONF["password"],db=SQL_CONF["database"],charset="utf8mb4",cursorclass=pymysql.cursors.DictCursor)
with conn.cursor() as cur:
    cur.execute("SELECT id, nombre, bio, img, aparicion, casa FROM heroes_ds"); heroes=list(cur.fetchall())
    cur.execute("SELECT id, nombre FROM peliculas_ds"); peliculas=list(cur.fetchall())
    cur.execute("""SELECT id, nombre, correo, password, img, rol, estado, google, fecha_creacion, fecha_actualizacion FROM usuarios_ds"""); usuarios=list(cur.fetchall())
    cur.execute("SELECT idmultimedia AS id, nombre, url, tipo FROM multimedias_ds"); multimedias=list(cur.fetchall())
    cur.execute("""SELECT id, papel, fecha_participacion, heroes_id, peliculas_id FROM protagonistas_ds"""); protagonistas=list(cur.fetchall())
    cur.execute("SELECT heroes_id, idmultimedia FROM multimedias_heroe_ds"); hero_media=list(cur.fetchall())
conn.close()
{"heroes":len(heroes),"peliculas":len(peliculas),"usuarios":len(usuarios),"multimedias":len(multimedias),"protagonistas":len(protagonistas),"hero_media_links":len(hero_media)}

## Transformar a documentos NoSQL

In [ ]:
pel_by_id={p["id"]:p for p in peliculas}; media_by_id={m["id"]:m for m in multimedias}
media_by_hero={}
for hm in hero_media:
    media_by_hero.setdefault(hm["heroes_id"],[]).append(media_by_id.get(hm["idmultimedia"]))
part_by_hero={}
for pr in protagonistas:
    pel=pel_by_id.get(pr["peliculas_id"],{"id":pr["peliculas_id"],"nombre":None})
    part_by_hero.setdefault(pr["heroes_id"],[]).append({"pelicula_id":pel["id"],"pelicula_nombre":pel["nombre"],"papel":pr["papel"],"fecha_participacion":to_iso_date(pr["fecha_participacion"])})
heroes_docs=[{"_id":int(h["id"]),"nombre":h["nombre"],"bio":h.get("bio"),"img":h.get("img"),"aparicion":to_iso_date(h.get("aparicion")),"casa":h.get("casa"),"peliculas":part_by_hero.get(h["id"],[]),"multimedias":media_by_hero.get(h["id"],[])} for h in heroes]
peliculas_docs=[{"_id":int(p["id"]),"nombre":p["nombre"]} for p in peliculas]
usuarios_docs=[{"_id":int(u["id"]),"nombre":u["nombre"],"correo":u["correo"],"password":u["password"],"img":u.get("img"),"rol":u["rol"],"estado":to_bool(u.get("estado")),"google":to_bool(u.get("google")),"fecha_creacion":to_iso_date(u.get("fecha_creacion")),"fecha_actualizacion":to_iso_date(u.get("fecha_actualizacion"))} for u in usuarios]
multimedias_docs=[{"_id":int(m["id"]),"nombre":m["nombre"],"url":m.get("url"),"tipo":m.get("tipo")} for m in multimedias]
protagonistas_docs=[{"_id":int(pr["id"]),"papel":pr["papel"],"fecha_participacion":to_iso_date(pr["fecha_participacion"]),"heroes_id":int(pr["heroes_id"]),"peliculas_id":int(pr["peliculas_id"])} for pr in protagonistas]
{"heroes":len(heroes_docs),"peliculas":len(peliculas_docs),"usuarios":len(usuarios_docs),"multimedias":len(multimedias_docs),"protagonistas":len(protagonistas_docs)}

## Cargar en MongoDB (o exportar JSON si no hay URI)

In [ ]:
class MongoSink:
    def __init__(self, uri, dbname):
        if MongoClient is None:
            raise RuntimeError("Falta pymongo. pip install pymongo")
        self.client=MongoClient(uri); self.db=self.client[dbname]
    def write(self, coll, docs):
        if not docs: return
        ops=[ReplaceOne({"_id":d["_id"]} if "_id" in d else {"nombre":d.get("nombre")}, d, upsert=True) for d in docs]
        self.db[coll].bulk_write(ops, ordered=False)
    def close(self):
        try: self.client.close()
        except: pass

class JSONSink:
    def __init__(self, outdir="output"):
        os.makedirs(outdir, exist_ok=True); self.outdir=outdir
    def write(self, coll, docs):
        with open(os.path.join(self.outdir, f"{coll}.json"),"w",encoding="utf-8") as f:
            json.dump(docs,f,ensure_ascii=False,indent=2)
    def close(self): pass

sink = MongoSink(MONGO_URI, MONGO_DB) if MONGO_URI else JSONSink("output")
for name,docs in [("heroes",heroes_docs),("peliculas",peliculas_docs),("usuarios",usuarios_docs),("multimedias",multimedias_docs),("protagonistas",protagonistas_docs)]:
    sink.write(name, docs)
sink.close()
print("[OK] Migración terminada.")